In [2]:
from dataservice import DataService
from IPython.display import Markdown, display
import seaborn as sns 
import matplotlib.pyplot as plt 
import pandas as pd 
from datetime import * 
import re
import plotly.express as px 

In [1]:
obj = DataService()
obj.set_source("https://my.raceresult.com/309137/")
obj.handle_cookie_banner() 
obj.click_show_all_buttons() 
obj.create_soup() 
obj.get_metadata() 
obj.expand_table() 
obj.get_table_as_df() 
data = obj.dataFrame

NameError: name 'DataService' is not defined

In [3]:
race = DataService()

df = race.scrape_data("https://my.raceresult.com/309137/")

[DEBUG] DataService initialized.
[DEBUG] Navigated to URL: https://my.raceresult.com/309137/
[DEBUG] Source set to: https://my.raceresult.com/309137/
[DEBUG] Current source: https://my.raceresult.com/309137/
Cookie-Banner geschlossen.


/Users/janikwahrheit/Library/CloudStorage/OneDrive-Persönlich/05_Code/Python/Webscraping_Race/dataservice.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.dataFrame["datetime"] = pd.to_datetime(self.dataFrame["datetime"])


In [4]:
df.columns

Index(['Pl.', 'Snr.', 'Name', 'Jg.', 'Verein', 'AK-Pl.', 'MW-Pl.', 'Zeit',
       'datetime', 'time_delta', 'time_minutes', 'AK', 'M/W', 'Age',
       'AG_numeric'],
      dtype='object')

In [16]:
# Assuming 'gender_counts' is already calculated (counts of 'M' and 'W')
fig = px.histogram(
    df,
    x="AK", 
    title = "Teilnehmer pro Altersklasse", 
    color_discrete_sequence=["#b81212"]
    
)

fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),  # Title font style
    xaxis_title = "Altersklasse", 
    yaxis_title = "Teilnehmer",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),  # Add gridlines
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white",  # Background color# Gap between bars
)

fig.update_traces(
    marker=dict(
        line=dict(width=1, color="black")  # Add border to bars
    ),
    opacity=0.8, 
    hovertemplate="<b>AK:</b> %{x}<br>" +  # Customize hover text
              "<b>Teilnehmer:</b> %{y}<extra></extra>" 
)


# Show the pie chart
fig.show()


In [6]:
import plotly.express as px
import datetime as dt

#for index, time in enumerate(df["Zeit"]): 
#    df["Zeit"].iloc[index] = dt.datetime.combine(dt.date(2025,1,1), time)

# Create the histogram
fig = px.histogram(
    df, 
    x="datetime", 
    nbins=200, 
    title="Verteilung der Rennzeiten",  # Add a title
    color_discrete_sequence=["#b81212"] # Custom color
)

# Customize layout
fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),  # Title font style
    xaxis_title = "Zeit in Minuten", 
    yaxis_title = "Anzahl",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),  # Add gridlines
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white",  # Background color
    bargap=0.2,  # Gap between bars
    xaxis_tickformat = "%H:%M:%S"
)


# Customize bars
fig.update_traces(
    marker=dict(
        line=dict(width=1, color="black")  # Add border to bars
    ),
    opacity=0.8, 
    hovertemplate="<b>Zeit in Minuten:</b> %{x}<br>" +  # Customize hover text
              "<b>Anzahl:</b> %{y}<extra></extra>" 
)

# Show the plot
fig.show()


In [15]:
boxplot_race_time = px.box(
  df, 
  y = "Zeit", 
  title = "Boxplot der Rennzeiten", 
  color_discrete_sequence=["#b81212"]  
)

# Customize layout
boxplot_race_time.update_layout(
    title_font=dict(size=20, family="Arial", color="black"), 
    yaxis_title="Zeit in Minuten",  # Y-axis title
    xaxis=dict(showgrid=True, gridcolor="lightgray"),  
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white"
)


boxplot_race_time.show() 

In [7]:
import plotly.express as px

# Assuming 'data' is your DataFrame
# Count the occurrences of 'M' and 'W' in the "M/W" column
gender_counts = df['M/W'].value_counts()


# Assuming 'gender_counts' is already calculated (counts of 'M' and 'W')
fig = px.pie(
    names=gender_counts.index,  # The categories (M and W)
    values=gender_counts.values,  # The count of each category
    title="Proportion of Male/Female Participants",  # Title of the pie chart
    color=gender_counts.index,  # Color by gender categories (M and W)
    color_discrete_map={"M": "#b81212", "W": "#ffffff"}
)

# Add a red border around the white sector (Female category)
fig.update_traces(
     hovertemplate="<b>Geschlecht: </b> %{label}<br>" +  # Display M/W category
                  "<b>Teilnehmer:</b> %{value} <extra></extra>",  # Display the count value
    marker=dict(
        line=dict(
            color="#b81212",  # Red color for the border
            width=1  # Set the width of the border
        )
    )
)

# Show the pie chart
fig.show()




In [22]:
df["Zeit"].dtype


dtype('O')

In [4]:
fig = px.scatter(
    df,
    y="time_minutes",  
    x="Age",  
    color="M/W", 
    title="Zusammenhang zwischen Alter und Zeit",
    color_discrete_map={"M": "#b81212", "W": "#000000"}, 
    trendline="ols"
)

fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),  
    yaxis_title="Zeit", 
    xaxis_title="Alter",
    yaxis=dict(
        showgrid=True, gridcolor="lightgray", tickformat="%H:%M:%S"  # Format time
    ),
    xaxis=dict(
        showgrid=True, gridcolor="lightgray"
    ),
    plot_bgcolor="white",  # Background color
)

fig.show()

In [7]:
import statsmodels.api as sm

filtered = df[df["M/W"] == "M"]

# Fit the OLS regression model
X = sm.add_constant(filtered["Age"])  # Adding a constant for the intercept
y = filtered["time_minutes"]
model = sm.OLS(y, X).fit()

# Summary of the regression
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:           time_minutes   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     313.7
Date:                Fri, 10 Jan 2025   Prob (F-statistic):           2.12e-60
Time:                        22:14:08   Log-Likelihood:                -3071.4
No. Observations:                 897   AIC:                             6147.
Df Residuals:                     895   BIC:                             6156.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         32.6486      0.809     40.379      0.0

In [33]:
from statsmodels.formula.api import ols

df.rename()

# Create the regression model including "M/W"
regression_model = ols("time_minutes ~ Age + C(Gender)", data=df)

# Fit the model
regression_model = regression_model.fit()

# Print the summary
print(regression_model.summary())


                            OLS Regression Results                            
Dep. Variable:           time_minutes   R-squared:                       0.303
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     290.0
Date:                Sat, 11 Jan 2025   Prob (F-statistic):          2.67e-105
Time:                        09:31:48   Log-Likelihood:                -4624.9
No. Observations:                1337   AIC:                             9256.
Df Residuals:                    1334   BIC:                             9271.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     33.3203      0.717     46.453      0.0

In [ ]:
import statsmodels.api as sm

# Manually handle categorical variables
df["Gender"] = df["Gender"].cat.codes  # Convert categories to integers (0, 1)

X = sm.add_constant(df[["Age", "Gender"]])  # Add constant for intercept
y = df["time_minutes"]

# Fit the OLS regression model
model = sm.OLS(y, X).fit()

# Summary of the regression
print(model.summary())
print

                            OLS Regression Results                            
Dep. Variable:           time_minutes   R-squared:                       0.303
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     290.0
Date:                Sat, 11 Jan 2025   Prob (F-statistic):          2.67e-105
Time:                        09:27:05   Log-Likelihood:                -4624.9
No. Observations:                1337   AIC:                             9256.
Df Residuals:                    1334   BIC:                             9271.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         33.3203      0.717     46.453      0.0